# TRABAJO PRÁCTICO Nº1 - GRUPO 9

Integrantes:
- Pablo Arias
- Manuel Chara
- Melissa Montes
- Sheila Aguilar
- Marcos Kippes
- Sergio Flieg
- Camila Danza
- Roberto Doello
- Sergio Antozzi

# Carga de librerias y Dataset

En esta sección importamos las librerías necesarias para nuestro análisis y el dataset sobre el cual que trabajaremos

In [240]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn  as sbn
import re

In [241]:
df_original = pd.read_csv('C:\\Users\\cdanz\\Documents\\DH\\properatti.csv')
data=df_original.copy()

# Analisis preliminar del Dataset

In [242]:
#Observamos las caracteristicas del DF
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  121220 non-null  int64  
 1   operation                   121220 non-null  object 
 2   property_type               121220 non-null  object 
 3   place_name                  121197 non-null  object 
 4   place_with_parent_names     121220 non-null  object 
 5   country_name                121220 non-null  object 
 6   state_name                  121220 non-null  object 
 7   geonames_id                 102503 non-null  float64
 8   lat-lon                     69670 non-null   object 
 9   lat                         69670 non-null   float64
 10  lon                         69670 non-null   float64
 11  price                       100810 non-null  float64
 12  currency                    100809 non-null  object 
 13  price_aprox_lo

In [243]:
#Creamos una funcion para ver el porcentaje de nulos por columna del Dataframe y
#las ordene de mayor a menor

def imprime_porcentaje_nulos_dataframe(df):
    null_percentage = df.isnull().mean() * 100
    null_percentage = null_percentage.sort_values(ascending=False)
    return null_percentage


imprime_porcentaje_nulos_dataframe(data)

floor                         93.483749
expenses                      88.234615
rooms                         60.905791
price_usd_per_m2              43.394654
lat-lon                       42.525986
lat                           42.525986
lon                           42.525986
surface_total_in_m2           32.443491
price_per_m2                  27.686850
currency                      16.837981
price_aprox_local_currency    16.837156
price                         16.837156
price_aprox_usd               16.837156
surface_covered_in_m2         16.422208
geonames_id                   15.440521
image_thumbnail                2.567233
place_name                     0.018974
description                    0.001650
operation                      0.000000
state_name                     0.000000
country_name                   0.000000
place_with_parent_names        0.000000
property_type                  0.000000
properati_url                  0.000000
title                          0.000000


In [244]:
# Controlamos si hay filas duplicadas en el DataFrame
numero_filas_duplicadas = data.duplicated().sum()

# Imprimir el resultado
print("Número de filas duplicadas en el DataFrame:", numero_filas_duplicadas)

Número de filas duplicadas en el DataFrame: 0


# Limpieza de datos

Del análisis de los valores unicos de cada columna y tras observar el porcentaje de nulos de cada una seleccionamos las columnas a eliminar.


*   ***Operation/Country_name***: las filas contienen un único valor

*   ***Expenses/Floor***: contienen un alto porcentaje de valores nulos que serían dificiles de imputar

*   **Unnamed: 0/properati_url:** no aportan información relevante

*   **Currency/price/price_per_m2:** la columna price cuenta con valores en pesos y en dolares sin criterio alguno, consideramos mas facil trabajar con las columnas price_aprox_in_usd y price_aprox_local_currency

*   **Lat-lon/Lat/Lon/Geonames_id:** se intentó cruzar datos con la db de geonames, para ver si se podian imputar nulos, pero no fue significativa la mejora. Se omite dicho analisis en el presente tp, con fines de resumir. La ubicación puede tratarse con las columna place_name.

Si bien hay más columnas que podrían eliminarse, y entendemos que este dataset no va a usarse realmente para un modelo de ML, como la premisa del trabajo es preparar el dataset para su uso en un modelo de ML consideramos que es el mismo modelo es el que decide que variables de entrada son importantes o no.





In [245]:
#Seleccionamos las columnas a dropear
columnas_a_eliminar = ['Unnamed: 0', 'operation', 'properati_url','country_name',
                       'floor','expenses','lon','price','currency','lat','lat-lon',
                       'geonames_id']

# Utilizamos el método drop para eliminar las columnas
data_clean = data.drop(columnas_a_eliminar, axis=1)


## Reducción de valores nulos y atípicos

Luego creamos dos funciones para controlar el porcentaje de valores nulos y los valores únicos en columnas. Ambas nos permiten observar si el código y las funciones que aplicamos están funcionando comoo esperamos.  

In [246]:
#Para controlar los valores unicos de las columnas
check_column_vc= lambda df, column: (data_clean[column].value_counts())

#Para controlar el porcentaje de valores nulos de las columnas
check_nulls = lambda df, column: (data_clean[column].isnull().sum() / len(df)) * 100


### Surface

Nuestra idea para tratar surface covered era calcular las medias de surface para cada valor unico de property type. Previo a ello, y considerando que la media es sensible a los valores extremos, calculamos los outliers de la columna según el property tipe. Agrupamos por property_type ya que distintos tipos de propiedades tienen dimensiones diferentes y por lo tanto sus outliers seran diferentes.

In [247]:
#Controlamos el numero de nulos previo a nuestra intervencion

#Para surface covered
print('Porcentaje de valores nulos para surface_covered:')
print(check_nulls(data_clean,'surface_covered_in_m2'))

#Para surface total
print('Porcentaje de valores nulos para surface_total:')
print(check_nulls(data_clean, 'surface_total_in_m2'))

Porcentaje de valores nulos para surface_covered:
16.422207556508827
Porcentaje de valores nulos para surface_total:
32.44349117307375


Aplicamos el IQR agrupando por property_type

In [248]:

# Crear un nuevo DataFrame para almacenar los valores sin outliers
df_no_outliers_surface = pd.DataFrame()

# Iterar sobre cada tipo de propiedad
for property_type, group in data_clean.groupby('property_type'):
    # Calcular los límites para detectar outliers en surface_covered_in_m2
    q1_covered = group['surface_covered_in_m2'].quantile(0.25)
    q3_covered = group['surface_covered_in_m2'].quantile(0.75)
    iqr_covered = q3_covered - q1_covered
    lower_bound_covered = q1_covered - 1.5 * iqr_covered
    upper_bound_covered = q3_covered + 1.5 * iqr_covered

    # Calcular los límites para detectar outliers en surface_total_in_m2
    q1_total = group['surface_total_in_m2'].quantile(0.25)
    q3_total = group['surface_total_in_m2'].quantile(0.75)
    iqr_total = q3_total - q1_total
    lower_bound_total = q1_total - 1.5 * iqr_total
    upper_bound_total = q3_total + 1.5 * iqr_total

    # Filtrar los datos dentro de los límites para surface_covered_in_m2 y surface_total_in_m2
    filtered_data_covered = group[(group['surface_covered_in_m2'] >= lower_bound_covered) & (group['surface_covered_in_m2'] <= upper_bound_covered)]
    filtered_data_total = group[(group['surface_total_in_m2'] >= lower_bound_total) & (group['surface_total_in_m2'] <= upper_bound_total)]

    # Agregar los datos filtrados al nuevo DataFrame
    df_no_outliers_surface = pd.concat([df_no_outliers_surface, filtered_data_covered, filtered_data_total])

# En versiones anteriores este código nos generaba valores duplicados, así que procedemos a dropearlos
df_no_outliers_surface= df_no_outliers_surface.drop_duplicates()

# Reemplazar el DataFrame original con el DataFrame limpio
data_clean= df_no_outliers_surface
#data_clean

In [249]:
#Controlamos el numero de nulos tras nuestra intervencion

#Para surface covered
print('Porcentaje de valores nulos para surface_covered:')
print(check_nulls(data_clean,'surface_covered_in_m2'))

#Para surface total
print('Porcentaje de valores nulos para surface_total:')
print(check_nulls(data_clean, 'surface_total_in_m2'))

Porcentaje de valores nulos para surface_covered:
6.629451433144329
Porcentaje de valores nulos para surface_total:
24.64988728078308


In [250]:

# Calculamos las diferencias por tipo de propiedad
data_clean['difference_by_property'] = data_clean.groupby('property_type')['surface_total_in_m2'].transform('mean') - data_clean.groupby('property_type')['surface_covered_in_m2'].transform('mean')

# Imputamos los valores faltantes en 'surface_covered_in_m2' usando la diferencia por propiedad
data_clean['surface_covered_in_m2'].fillna(data_clean['surface_total_in_m2'] - data_clean['difference_by_property'], inplace=True)

# Imputamos los valores faltantes en 'surface_total_in_m2' usando la diferencia por propiedad
data_clean['surface_total_in_m2'].fillna(data_clean['surface_covered_in_m2'] + data_clean['difference_by_property'], inplace=True)

# Elimina la columna intermedia utilizada para la imputación
data_clean.drop('difference_by_property', axis=1, inplace=True)

# Ahora, 'surface_covered_in_m2' y 'surface_total_in_m2' contienen los valores imputados considerando la diferencia por tipo de propiedad

In [251]:
#Controlamos el numero de nulos tras nuestra intervencion

#Para surface covered
print('Porcentaje de valores nulos para surface_covered:')
print(check_nulls(data_clean,'surface_covered_in_m2'))

#Para surface total
print('Porcentaje de valores nulos para surface_total:')
print(check_nulls(data_clean, 'surface_total_in_m2'))

Porcentaje de valores nulos para surface_covered:
0.0
Porcentaje de valores nulos para surface_total:
0.0


### Rooms

Decidimos mantener rooms a pesar de su alto porcentaje de valores nulos (60%) y de valores atípicos porque consideramos que el numero de ambientes es un factor que incide en el valor de la propiedad.

Para disminuir los nulos primero intentamos obtener el numero de ambientes faltantes de las columnas'description' y 'title'.

In [252]:
ambientes = {
    '1': 'uno',
    '2': 'dos',
    '3': 'tres',
    '4': 'cuatro',
    '5': 'cinco',
    '6': 'seis',
    '7': 'siete',
    '8': 'ocho',
    '9': 'nueve',
    '10': 'diez',
}

# Creamos una funcion para encontrar el numero de ambientes en description y title para reducir los valores nulos
def extract_rooms(row):
    if pd.notnull(row['rooms']):
        return row['rooms']

    match = re.search(r'(\d+|' + '|'.join(ambientes.values()) + ') (ambientes?|ambs?|amb\.) ',
                      str(row['title']) + ' ' + str(row['description']), flags=re.IGNORECASE)
    
    if match:
        matched_number = match.group(1)
        return int(matched_number) if matched_number.isdigit() else None
    else:
        return None

print("Antes de extract_rooms:")
print(data_clean['rooms'].head(10))

data_clean['rooms'] = data_clean.apply(extract_rooms, axis=1)

print("\nDespués de extract_rooms:")
print(data_clean['rooms'].head(10))

print('\nPorcentaje de valores nulos tras extracción de valores:')
check_nulls(data_clean,'rooms')

Antes de extract_rooms:
0      NaN
6      NaN
16     4.0
34     6.0
46     4.0
80     4.0
94     3.0
111    NaN
116    NaN
137    6.0
Name: rooms, dtype: float64



Después de extract_rooms:
0      2.0
6      NaN
16     4.0
34     6.0
46     4.0
80     4.0
94     3.0
111    NaN
116    NaN
137    6.0
Name: rooms, dtype: float64

Porcentaje de valores nulos tras extracción de valores:


44.94910556569432

In [253]:
#Chequeamos los valores unicos de la columna rooms tras aplicar extract_rooms
check_column_vc(data_clean,'rooms')

rooms
2.0        16911
3.0        16413
4.0         9124
1.0         7967
5.0         3562
6.0         1180
7.0          534
8.0          242
10.0         157
9.0          125
11.0          53
12.0          35
13.0          21
15.0          19
14.0          13
17.0          10
22.0           5
16.0           5
20.0           5
21.0           4
19.0           3
18.0           3
5151.0         3
25.0           2
23.0           2
20173.0        1
2018.0         1
32.0           1
4000.0         1
72.0           1
30.0           1
20174.0        1
24.0           1
103.0          1
2017.0         1
27.0           1
Name: count, dtype: int64

Observamos que hay valores imposibles para la variable medida. Para deshacernos de outliers tomamos 10 ambientes como el maximo posible de ambientes ya que aplicar el rango interquartil dejaria muchos valores afuera

In [254]:

# Convertir la columna 'rooms' a tipo numérico (float) y manejar valores no numéricos como NaN
data_clean['rooms'] = pd.to_numeric(data_clean['rooms'], errors='coerce')

# Filtrar filas con 'rooms' mayores a 10
rows_gt_10 = data_clean[data_clean['rooms'] > 10]

# Conservar solo filas con 'rooms' menores o iguales a 10 y nulos
data_clean = data_clean[~data_clean.index.isin(rows_gt_10.index)]

# Imprimir el número de filas después de la operación
print("Número de filas después de la operación:", data_clean.shape[0])

#Controlamos que los valores resultantes cumplan con el criterio
check_column_vc(data_clean,'rooms')

Número de filas después de la operación: 102273


rooms
2.0     16911
3.0     16413
4.0      9124
1.0      7967
5.0      3562
6.0      1180
7.0       534
8.0       242
10.0      157
9.0       125
Name: count, dtype: int64

In [255]:
check_nulls(data_clean,'rooms')

45.03436879723876

A continuación imputamos los valores nulos restantes. Para eso, primero calculamos el promedio de la superficie cubierta por cada valor unico de property_type y rooms. Es decir, establecemos que para cada tipo de propiedad, como por ejemplo un departamento, habra 10 medias de superficie, uno por cada posible numero de rooms.

Una vez contamos con estos promedios,identificamos las filas nulas de rooms y tomamos el property type y la superficie cubierta de esas filas. Tras identificar el property type comparamos el promedio de superficie de la fila con los promedios de surface que establecimos antes. Calculamos a qué media de superficie se acerca mas la superficie de la fila y en base a eso se asigna en rooms el valor de rooms asociado a esa media de superficie.

In [256]:
#Tomando las filas completas en las columnas rooms, property_type y surface_total_in_m2
# Calcular el promedio de surface_total_in_m2 para cada valor único de la columna rooms y property_type

#Calculamos el promedio y creamos un diccionario para gaurdar los valores
promedio_surface_covered_by_rooms = data_clean.groupby(['property_type', 'rooms'])['surface_covered_in_m2'].mean().to_dict()

# Creamos una función para completar los nulos en rooms en base al promedio de surface_total_in_m2 agrupado por rooms y property_type
def fill_rooms(row):
    if pd.notnull(row['rooms']):
        return row['rooms']

    property_type = row['property_type']
    # Filtrar el diccionario por property_type
    property_type_means = {k[1]: v for k, v in promedio_surface_covered_by_rooms.items() if k[0] == property_type}

    if property_type_means:
        closest_mean = min(
            property_type_means.items(),
            key=lambda x: abs(row['surface_total_in_m2'] - x[1])
        )
        return closest_mean[0]  # Tomamos el valor correspondiente a 'rooms'

    # Si no hay valores para property_type, devolvemos NaN
    return np.nan

# Aplicamos la función fill_rooms para completar los nulos en la columna rooms
data_clean['rooms'] = data_clean.apply(lambda row: fill_rooms(row) if pd.isnull(row['rooms']) else row['rooms'], axis=1)

check_nulls(data_clean, 'rooms')


C:\Users\cdanz\AppData\Local\Temp\ipykernel_7032\4141145005.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['rooms'] = data_clean.apply(lambda row: fill_rooms(row) if pd.isnull(row['rooms']) else row['rooms'], axis=1)


0.0

### Place name

Para imputar los valores faltantes en place_name tomamos la columna place_name_with_parent_names.

In [257]:
def extract_place_name(row):
    # Eliminar pipes y espacios adicionales y luego dividir la cadena
    places = re.sub(r'\s*\|\s*', '|', str(row['place_with_parent_names'])).split('|')

    if len(places) >= 3:
        return places[2].strip()
    else:
        return None  # o cualquier valor predeterminado que desees para casos donde no hay un tercer elemento

# Llenar los valores nulos en 'place_name' directamente en el DataFrame
data_clean['place_name'].fillna(data_clean.apply(extract_place_name, axis=1), inplace=True)

check_nulls(data_clean,'place_name')

C:\Users\cdanz\AppData\Local\Temp\ipykernel_7032\396764107.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['place_name'].fillna(data_clean.apply(extract_place_name, axis=1), inplace=True)


0.0

In [258]:
#Dropeamos la columna place_with_parent_names

data_clean.drop('place_with_parent_names',axis=1, inplace=True)

C:\Users\cdanz\AppData\Local\Temp\ipykernel_7032\3290184769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean.drop('place_with_parent_names',axis=1, inplace=True)


### Columnas de price

In [259]:
data_clean.columns

Index(['property_type', 'place_name', 'state_name',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'rooms',
       'description', 'title', 'image_thumbnail'],
      dtype='object')

In [260]:
#dropeamos price_usd_per_m2 y después la volvemos a agregar como división entre columnas de superficie y precio.
# Además dropeamos las demas columnas con precios para no generar confusión. Dejamos solo columnas en dolares.
data_clean.drop(['price_aprox_local_currency',"price_per_m2","price_usd_per_m2"],axis=1, inplace=True)

C:\Users\cdanz\AppData\Local\Temp\ipykernel_7032\1402158362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean.drop(['price_aprox_local_currency',"price_per_m2","price_usd_per_m2"],axis=1, inplace=True)


In [261]:
check_nulls(data_clean,'price_aprox_usd')

13.865829691120823

In [262]:

# Calcular la media por lugar y tipo de propiedad
mean_price_usd= data_clean.groupby(['place_name', 'property_type'])['price_aprox_usd'].median().to_dict()

# Función para llenar valores nulos en 'price_usd_per_m2'
def fill_price_usd(row):
    if pd.notnull(row['price_aprox_usd']):
        return row['price_aprox_usd']
    else:
        key = (row['place_name'], row['property_type'])
        return mean_price_usd.get(key)

# Aplicar la función fill_price_per_m2 para llenar los valores nulos en 'price_usd_per_m2'
data_clean['price_aprox_usd'] = data_clean.apply(fill_price_usd, axis=1)

# Verificar los valores nulos en 'price_usd_per_m2'
check_nulls(data_clean, 'price_aprox_usd')

C:\Users\cdanz\AppData\Local\Temp\ipykernel_7032\1676889598.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['price_aprox_usd'] = data_clean.apply(fill_price_usd, axis=1)


0.2825770242390465

In [263]:
#Agrego price_usd_per_m2 de nuevo como división de columnas completas
data_clean["price_usd_per_m2"]=data_clean["price_aprox_usd"]/data_clean["surface_total_in_m2"]



C:\Users\cdanz\AppData\Local\Temp\ipykernel_7032\3937847477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean["price_usd_per_m2"]=data_clean["price_aprox_usd"]/data_clean["surface_total_in_m2"]


In [264]:
data_clean.columns

Index(['property_type', 'place_name', 'state_name', 'price_aprox_usd',
       'surface_total_in_m2', 'surface_covered_in_m2', 'rooms', 'description',
       'title', 'image_thumbnail', 'price_usd_per_m2'],
      dtype='object')

### Columnas que siguen con valores nulos

In [265]:
#Dopreamos las filas nulas que nos quedaron

data_clean['image_thumbnail'].dropna(inplace=True)
data_clean['price_aprox_usd'].dropna(inplace=True)
data_clean['price_usd_per_m2'].dropna(inplace=True)
data_clean['description'].dropna(inplace=True)


C:\Users\cdanz\AppData\Local\Temp\ipykernel_7032\2419034427.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['image_thumbnail'].dropna(inplace=True)
C:\Users\cdanz\AppData\Local\Temp\ipykernel_7032\2419034427.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['price_aprox_usd'].dropna(inplace=True)
C:\Users\cdanz\AppData\Local\Temp\ipykernel_7032\2419034427.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d

# Análisis de resultados

In [266]:
# Calcular el porcentaje de filas remanentes respecto al DataFrame original
remaining_rows_percentage = (data_clean.shape[0] / data.shape[0]) * 100

print(f"Porcentaje de filas remanentes respecto al DataFrame original: {remaining_rows_percentage}%")


data_clean.to_csv('properati_cleaned.csv', index=False)



Porcentaje de filas remanentes respecto al DataFrame original: 84.36974096683716%


In [269]:
#Vemos los cambios en el porcentaje de nulos tras las operaciones realizadas
#Si se ejecuta dos veces muestra los porcentajes en 0
imprime_porcentaje_nulos_dataframe(data_clean)

property_type            0.0
place_name               0.0
state_name               0.0
price_aprox_usd          0.0
surface_total_in_m2      0.0
surface_covered_in_m2    0.0
rooms                    0.0
description              0.0
title                    0.0
image_thumbnail          0.0
price_usd_per_m2         0.0
dtype: float64

In [268]:
#Aca luego de imputar los nulos en funcion del dataframe de agregacion (media)
#diminuye considerablemente los nulos en las columnas price,
#como es un porcentaje muy pequeño los dropeamos.

data_clean.dropna(inplace=True)

print("El resultado de la limpieza es el siguiente\n")
print(data_clean.shape)

El resultado de la limpieza es el siguiente

(99930, 11)


C:\Users\cdanz\AppData\Local\Temp\ipykernel_7032\2058273694.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean.dropna(inplace=True)
